In [3]:
# !pip install sacrebleu sacremoses
# !pip install evaluate

In [ ]:
# DATASET
from datasets import load_dataset, Dataset

## Load the dataset
ds = load_dataset("UWV/Leesplank_NL_wikipedia_simplifications_preprocessed", split="train", streaming=True)
ds = ds.shuffle(seed=42)

small_ds = ds.take(5)

In [16]:
small_ds = ds.take(5)

In [17]:
for idx in small_ds:
    print(idx)

{'instruction': 'Vereenvoudig: ', 'prompt': 'Montes Abalde leerde al op 10-jarige leeftijd in de muziekschool van de \\"Banda de Música Agrupación Musical Atlántida de Matamá\\" de trombone te bespelen. Vervolgens studeerde hij trombone en compositie aan het \\"Conservatorio de Música de Vigo\\". Hij behaalde zijn diploma\'s voor trombone in oktober 2005 en voor compositie in maart 2007. Later volgde hij ook de muziekopleiding aan de universiteit van Vigo. Naast zijn \\"Banda de Música Agrupación Musical Atlántida de Matamá\\" is hij ook lid in het \\"Orquestra Clásica de Vigo\\".', 'result': 'Toen Montes Abalde tien jaar oud was, begon hij trombone te spelen bij de muziekschool van de Banda de Música Agrupación Musical Atlántida de Matamá. Hij ging verder met zijn muziekstudie aan het Conservatorio de Música de Vigo, waar hij leerde hoe hij trombone moest spelen en hoe hij muziek kon componeren. In oktober 2005 kreeg hij zijn diploma voor trombone en in maart 2007 voor compositie. Daa

In [2]:
# SARI
from evaluate import load
sari = load("sari")

In [7]:
# INFERENCE
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "auto"
model_path = "ibm-granite/granite-3.0-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# drop device_map if running on CPU
model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [ ]:
# INFERENCE AND CALCULATE SARI SCORE
import re

row = df.iloc[10000]

print(row)

# Input and references
instruction = row['instruction']
input = row['prompt']
references = row['result']

# Run inference
chat = [
    { "role": "user", "content": f"{instruction} {input}" },
]
chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# tokenize the text
input_tokens = tokenizer(chat, return_tensors="pt").to('cpu')

# generate output tokens
output = model.generate(**input_tokens, 
                        max_new_tokens=100)

# Decode output
output = tokenizer.batch_decode(output)

instruction                                             Vereenvoudig: 
prompt                De originele versie bevatte de volgende spellen:
result               In de originele versie waren deze spellen inbe...
__index_level_0__                                              1905410
Name: 10000, dtype: object


In [22]:
output

['<|start_of_role|>user<|end_of_role|>Vereenvoudig:  De originele versie bevatte de volgende spellen:<|end_of_text|>\n<|start_of_role|>assistant<|end_of_role|>1. Chess\n2. Checkers\n3. Backgammon\n4. Sudoku\n5. Tetris\n6. Chess\n7. Checkers\n8. Backgammon\n9. Sudoku\n10. Tetris\n\nDe originele versie bevatte de volgende spellen: Chess, Checkers, Backgammon, Sudoku, Tetris.<|end_of_text|>']

In [18]:
sari_score = sari.compute(sources=[input], predictions=output, references=[[references]])
sari_score

{'sari': 45.46374105197635}